##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用模板创建 TFX 流水线

注：我们建议在 Google Cloud Vertex AI Workbench 上运行本教程。[在 Vertex AI Workbench 上启动此笔记本](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb)。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/template"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/template.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/template.ipynb"> <img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/template.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

## 简介

本文档将说明如何使用 TFX Python 软件包提供的*模板*创建 TensorFlow Extended (TFX) 流水线。其中的许多说明为 Linux shell 命令，这些命令将在 AI Platform 笔记本实例上运行。本文档还提供了相应的 Jupyter 笔记本代码单元，它们使用 `!` 调用这些命令。

您将使用芝加哥市发布的 [Taxi Trips 数据集](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)构建流水线。我们强烈建议您尝试使用您自己的数据集构建流水线，并将此流水线用作基准。


## 第 1 步：设置环境

AI Platform Pipelines 将准备用于构建流水线的开发环境和用于运行新构建的流水线的 Kubeflow 流水线集群。

**注**：要选择特定的 TensorFlow 版本，或选择 GPU 实例，请在 AI Platform 笔记本中创建一个已预安装 TensorFlow 的实例。


安装具有 `kfp` 额外要求的 `tfx` Python 软件包。

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

我们来检查一下 TFX 的版本。

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

在 AI Platform Pipelines 中，TFX 使用 [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) 在托管的 Kubernetes 环境中运行。

我们来设置一些环境变量以使用 Kubeflow Pipelines。

首先，获取您的 GCP 项目 ID。

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

我们还需要访问您的 KFP 集群。您可以在 Google Cloud Console 的“AI Platform &gt; 流水线”菜单下访问它。可以从流水线信息中心的网址中找到 KFP 集群的“端点”，也可以从启动此笔记本的 Getting Started 页面的网址中获取。我们来创建一个 `ENDPOINT` 环境变量并将其设置为 KFP 集群端点。**ENDPOINT 应该仅包含网址的主机名部分**。例如，如果 KFP 信息中心的网址为 `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`，ENDPOINT 值将为 `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`。

> **注：您必须在下面设置 ENDPOINT 值。**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

在当前的 GCP 项目下，将镜像名称设置为 `tfx-pipeline`。

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

好了。我们已完成创建流水线的准备工作。

## 第 2 步：将预定义的模板复制到项目目录

在此步骤中，我们将通过复制预定义模板中的其他文件来创建工作流水线项目目录和文件。

您可以通过更改下面的 `PIPELINE_NAME` 为流水线使用其他名称。这也将成为存放文件的项目目录的名称。

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

TFX 的 TFX Python 软件包中包含 <code>taxi</code> 模板。如果您打算解决包括分类和回归在内的逐点预测问题，可以将此模板作为基础。

`tfx template copy` CLI 命令会将预定义的模板文件复制到您的项目目录中。

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

将此笔记本中的工作目录上下文更改为项目目录。

In [ ]:
%cd {PROJECT_DIR}

> 注：创建项目目录后，不要忘记点击进入项目目录来更改左侧 `File Browser` 中的目录。

## 第 3 步：浏览复制的源文件。

TFX 模板提供了用于构建流水线的基本基架文件，包括 Python 源代码、样本数据，以及用于分析流水线输出的 Jupyter 笔记本。`taxi` 模板使用与 [Airflow 教程](https://tensorflow.google.cn/tfx/tutorials/tfx/airflow_workshop)相同的*芝加哥出租车*数据集和 ML 模型。

以下是每个 Python 文件的简要说明。

- `pipeline` - 此目录包含流水线的定义
    - `constants.py` - 定义流水线运行程序的通用常量
    - `pipeline.py` - 定义 TFX 组件和流水线
- `models` - 此目录包含 ML 模型定义
    - `features.py`、`features_test.py` - 定义模型的特征
    - `preprocessing.py`、`preprocessing_test.py` - 使用 `tf::Transform` 定义预处理作业
    - `estimator` - 此目录包含一个基于 Estimator 的模型
        - `constants.py` - 定义模型的常量
        - `model.py`、`model_test.py` - 使用 TF Estimator 定义 DNN 模型
    - `keras` - 此目录包含一个基于 Keras 的模型
        - `constants.py` - 定义模型的常量
        - `model.py` 、`model_test.py` 使用 Keras 定义 DNN 模型
- `local_runner.py`,`kubeflow_dag_runner.py` - 为每个编排引擎定义运行程序


您可能注意到某些文件的名称中包含 `_test.py`。这些是流水线的单元测试，建议您在实现自己的流水线时添加更多单元测试。您可以通过提供带有 `-m` 标记的测试文件的模块名称来运行单元测试。通常，您可以通过删除 `.py` 扩展名并将 `/` 替换为 `.` 来获得模块名称。例如：

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 第 4 步：运行您的第一个 TFX 流水线。

TFX 流水线中的组件将以 [ML Metadata 工件](https://tensorflow.google.cn/tfx/guide/mlmd) 的形式为每个运行生成输出，并且需要将它们存储在某个位置。您可以使用 KFP 集群能够访问的任何存储，在本例中，我们将使用 Google Cloud Storage (GCS)。默认的 GCS 存储分区会自动创建，它的名称将为 `<your-project-id>-kubeflowpipelines-default`。


我们将样本数据上传到 GCS 存储分区，以便可以稍后在流水线中使用。

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

我们使用 `tfx pipeline create` 命令创建一个 TFX 流水线。

> 注：在为 KFP 创建流水线时，我们需要一个容器镜像，该镜像将用于运行流水线。`skaffold` 将为我们构建镜像。由于 Skaffold 会从 Docker Hub 拉取基础镜像，因此，当我们第一次构建镜像时，将花费 5~10 分钟，但从第二次构建开始，花费的时间会少得多。

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

创建流水线时，将生成 `Dockerfile` 来构建 Docker 镜像。不要忘记将它和其他源文件一起添加到您的源代码控制系统（例如 Git）。

注：如果未安装 `airflow` 且未指定 `--engine`， `kubeflow` 将被自动选作编排引擎。

现在，使用 `tfx run create` 命令以新创建的流水线开始执行运行。

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

或者，您也可以在 KFP 信息中心中运行流水线。新的执行运行将在 KFP 信息中心的“Experiments”下列出。点击进入实验后，您可以监视进度并呈现在执行运行期间创建的工件。

但是，我们建议您访问 KFP 信息中心。您可以从 Google Cloud Console 中的 Cloud AI Platform 流水线菜单访问 KFP 信息中心。进入信息中心后，您将能够找到流水线，并访问有关流水线的大量信息。例如，您可以在 *Experiments* 菜单下找到运行，当您在“Experiments”下打开执行运行后，可以在 *Artifacts* 菜单下找到您的所有工件。

> 注：如果流水线运行失败，您可以在 KFP 信息中心的“Experiments”标签页中查看每个 TFX 组件的详细日志。

失败的一个主要来源是权限相关问题。请确保您的 KFP 集群具有访问 Google Cloud API 的权限。您可以[在 GCP 中创建 KFP 集群时](https://cloud.google.com/ai-platform/pipelines/docs/setting-up)对此进行配置，或参阅 [GCP 中的问题排查](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting)文档。

## 第 5 步：添加用于数据验证的组件

在此步骤中，您将添加用于数据验证的组件，包括 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator`。如果您对数据验证感兴趣，请参阅 [TensorFlow Data Validation 使用入门](https://tensorflow.google.cn/tfx/data_validation/get_started)。

> **双击将目录更改为 `pipeline`，然后再次双击打开 `pipeline.py`**。查找并取消注释将 `StatisticsGen`、`SchemaGen` 和 `ExampleValidator` 添加到流水线的 3 行代码。（提示：搜索包含 `TODO(step 5):` 的注释）。编辑 `pipeline.py` 后请务必保存。

现在，您需要使用修改后的流水线定义更新现有流水线。使用 `tfx pipeline update` 命令更新流水线，然后使用 `tfx run create` 命令创建更新后的流水线的新执行运行。


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### 查看流水线输出

访问 KFP 信息中心并在流水线运行页面中找到流水线输出。点击左侧的 <em>Experiments</em> 标签页，然后在“Experiments”页面中点击 <em>All runs</em>。您应该能够在您的流水线下方找到最近的运行。

## 第 6 步：添加用于训练的组件

在此步骤中，您将添加用于模型训练和模型验证的组件，包括 `Transform`、`Trainer`、`Resolver`、`Evaluator` 和 `Pusher`。

> **双击打开 `pipeline.py`**。找到并取消注释将 `Transform`、`Trainer`、`Resolver`、`Evaluator` 和 `Pusher` 添加到流水线的 5 行代码。（提示：搜索 `TODO(step 6):`）

和之前一样，您现在需要使用修改后的流水线定义来更新现有流水线。指令与第 5 步相同。请使用 `tfx pipeline update` 更新流水线，然后使用 `tfx run create` 创建执行运行。


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

此执行运行成功完成后，您在 AI Platform Pipelines 中创建并运行第一个 TFX 流水线的操作也将完成！

**注**：如果我们更改了模型代码中的任何内容，我们也必须重建容器镜像。我们可以使用 `pipeline update` 命令中的 `--build-image` 标志来触发重建。

**NOTE:** You might have noticed that every time we create a pipeline run, every component runs again and again even though the input and the parameters were not changed. It is waste of time and resources, and you can skip those executions with pipeline caching. You can enable caching by specifying `enable_cache=True` for the `Pipeline` object in `pipeline.py`.


## 第 7 步：（*可选*）尝试 BigQueryExampleGen。

[BigQuery](https://cloud.google.com/bigquery) 是一种无服务器、扩缩能力极强且经济实惠的云数据仓库。BigQuery 可在 TFX 中用作训练样本的来源。在此步骤中，我们将向流水线添加 `BigQueryExampleGen`。

> **双击打开 `pipeline.py`**。注释掉 `CsvExampleGen` 并取消注释创建 `BigQueryExampleGen` 实例的行。您还需要取消注释 `create_pipeline` 函数的 `query` 参数。

我们需要指定用于 BigQuery 的 GCP 项目，为此，您需要在创建流水线时在 `beam_pipeline_args` 中设置 `--project`。

> **双击打开 `configs.py`**。取消注释 `GOOGLE_CLOUD_REGION`、`BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` 和 `BIG_QUERY_QUERY` 的定义。您应将此文件中的区域值替换为用于您的 GCP 项目的正确值。

> **注：您必须先在 `configs.py` 文件中设置您的 GCP 区域，然后才能继续。**

> **浏览到上一级目录。**点击文件列表上方的目录名称。该目录名称为流水线的名称，如果未进行更改，则为 `my_pipeline`。

> **双击打开 `kubeflow_runner.py`**。为 `create_pipeline` 函数取消注释 `query` 和 `beam_pipeline_args` 两个参数。

现在，流水线已准备好使用 BigQuery 作为样本源。像之前一样更新流水线，并像在第 5 和第 6 步中那样创建新的执行运行。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## 第 8 步：（*可选*）尝试带有 KFP 的 Dataflow。

多个 [TFX 组件使用 Apache Beam](https://tensorflow.google.cn/tfx/guide/beam) 来实现数据并行流水线，这意味着您可以使用 [Google Cloud Dataflow](https://cloud.google.com/dataflow/) 分布数据处理工作负载。在此步骤中，我们将设置 Kubeflow 编排器，以将 Dataflow 用作 Apache Beam 的数据处理后端。

> **双击 `pipeline` 更改目录，然后双击打开 `configs.py`**。取消注释 `GOOGLE_CLOUD_REGION` 和 `DATAFLOW_BEAM_PIPELINE_ARGS` 的定义。

> **浏览到上一级目录。**点击文件列表上方的目录名称。该目录名称为流水线的名称，如果未进行更改，则为 `my_pipeline`。

> **双击打开 `kubeflow_runner.py`**。取消注释 `beam_pipeline_args`。（还要确保注释掉您在第 7 步中添加的当前 `beam_pipeline_args`。）

现在，流水线已准备好使用 Dataflow。像在第 5 和第 6 步中的那样更新流水线并创建执行运行。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

您可以在 [Cloud Console 的 Dataflow](http://console.cloud.google.com/dataflow) 中找到您的 Dataflow 作业。


## 第 9 步：（*可选*）使用 KFP 尝试 Cloud AI Platform Training 和 Cloud AI Platform Prediction

TFX 可与多种代管式 GCP 服务（例如，[Cloud AI Platform Training 和 Cloud AI Platform Prediction](https://cloud.google.com/ai-platform/)）互操作。您可以设置 `Trainer` 组件以使用 Cloud AI Platform Training（一项用于训练 ML 模型的代管式服务）。此外，当您完成模型构建并准备应用时，可以将您的模型*推送*到 Cloud AI Platform Prediction 进行应用。在此步骤中，我们将设置 `Trainer` 和 `Pusher` 组件以使用 Cloud AI Platform 服务。

> 在编辑文件之前，您可能必须首先启用 *AI Platform Training &amp; Prediction API*。

> **双击 `pipeline` 更改目录，然后双击打开 `configs.py`**。取消注释 `GOOGLE_CLOUD_REGION`、`GCP_AI_PLATFORM_TRAINING_ARGS` 和 `GCP_AI_PLATFORM_SERVING_ARGS` 的定义。我们将使用自定义构建的容器镜像在 Cloud AI Platform Training 中训练模型，因此我们应将 `GCP_AI_PLATFORM_TRAINING_ARGS` 中的 `masterConfig.imageUri` 设置为与上文中 `CUSTOM_TFX_IMAGE` 相同的值。

> **浏览到上一级目录，然后双击打开 `kubeflow_runner.py`**。取消注释 `ai_platform_training_args` 和 `ai_platform_serving_args`。

像在第 5 和第 6 步中的那样更新流水线并创建执行运行。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

您可以在 [Cloud AI Platform 作业](https://console.cloud.google.com/ai-platform/jobs)中找到您的训练作业。如果流水线成功完成，则可以在 [Cloud AI Platform 模型](https://console.cloud.google.com/ai-platform/models)中找到您的模型。

## 第 10 步：将您的数据提取到流水线中

我们为使用芝加哥出租车数据集的模型创建了流水线。现在是时候将数据放入流水线中了。

您的数据可以存储在流水线能够访问的任何位置，包括 GCS 或 BigQuery。您将需要修改流水线定义以访问您的数据。

1. 如果您的数据存储在文件中，请修改 `kubeflow_runner.py` 或 `local_runner.py` 中的 `DATA_PATH`，并将其设置为文件的位置。如果您的数据存储在 BigQuery 中，请修改 `pipeline/configs.py` 中的 `BIG_QUERY_QUERY` 以正确查询您的数据。
2. 在 `models/features.py` 中添加特征。
3. 修改 `models/preprocessing.py` 以[转换输入数据用于训练](https://tensorflow.google.cn/tfx/guide/transform)。
4. 修改 `models/keras/model.py` 和 `models/keras/constants.py` 以[描述您的 ML 模型](https://tensorflow.google.cn/tfx/guide/trainer)。
    - 您也可以使用基于 Estimator 的模型。在 `pipeline/configs.py` 中将 `RUN_FN` 常量更改为 `models.estimator.model.run_fn`。

有关更多介绍，请参阅 [Trainer 组件指南](https://tensorflow.google.cn/tfx/guide/trainer)。

## 清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以通过访问每个控制台来清理各个资源：

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
